Importação das Bibliotecas

In [1]:
import os
import numpy as np
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
import multiprocessing
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from skimage.io import imread, imshow
from skimage.transform import resize
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, 
    BatchNormalization, Dropout, Lambda, Activation, MaxPool2D, Concatenate,
    RandomContrast
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-07-26 08:24:12.727600: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-26 08:24:12.868461: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-26 08:24:12.869281: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 08:24:13.676574: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Esse código é para a utilização de todos os processadores

In [2]:
num_cores = multiprocessing.cpu_count()
config = tf.compat.v1.ConfigProto(inter_op_parallelism_threads=num_cores,
                                  intra_op_parallelism_threads=num_cores)
tf.compat.v1.Session(config=config)


Inicialização das Variáveis que seram utilizadas dirante a execução do código

In [3]:
IMG_HEIGHT = 64
IMG_WIDTH = IMG_HEIGHT
IMG_CHANNELS = 2

batch_size = 10
epochs = 15

Carrega os dados de treinamento

In [4]:
# Defina o diretório onde os arquivos foram salvos
# path = "/home/cristiano/Imagens/"
path = "/home/emanoelcepr/Imagens/"

diretorio_imagens =  path + "imagens_originais/Sentinel-2/treino_processadas/"

# Caminhos completos para os arquivos X_train.npy e Y_train.npy
X_train_file = os.path.join(diretorio_imagens, 'X_train.npy')
Y_train_file = os.path.join(diretorio_imagens, 'Y_train.npy')

# Carregar os dados de volta para o notebook
X_train = np.load(X_train_file)
Y_train = np.load(Y_train_file)

# Verificar a forma (shape) das arrays carregadas
print('Shape de X_train:', X_train.shape)
print('Shape de Y_train:', Y_train.shape)


Shape de X_train: (2122, 64, 64, 2)
Shape de Y_train: (2122, 64, 64, 1)


Estrutura da Rede U-Net

In [5]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)  
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)  # Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation='sigmoid')(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

Cria e complita a rede

In [6]:
my_unet = build_unet(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
my_unet.compile(optimizer=Adam(learning_rate=1e-4),  loss='binary_crossentropy', metrics=['accuracy'])

Visualização da estrutura criada

In [7]:
print(my_unet.summary())

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 64, 64, 2)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 64, 64, 64)           1216      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 64, 64, 64)           256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 64, 64, 64)           0         ['batch_normalization[0][0

Inicializa o treinamento e salva o resultado 

In [8]:
filepath = diretorio_imagens + "model70-30.h5"
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [earlystopper, checkpoint]
history = my_unet.fit(X_train, Y_train, validation_split=0.2, batch_size=batch_size, epochs=epochs, callbacks=callbacks_list)

Epoch 1/15
170/170 [==============================] - ETA: 0s - loss: 0.2304 - accuracy: 0.9656
Epoch 1: val_loss improved from inf to 0.36421, saving model to /home/emanoelcepr/Imagens/imagens_originais/Sentinel-2/treino_processadas/model70-30.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


170/170 [==============================] - 164s 939ms/step - loss: 0.2304 - accuracy: 0.9656 - val_loss: 0.3642 - val_accuracy: 0.9277
Epoch 2/15
170/170 [==============================] - ETA: 0s - loss: 0.0995 - accuracy: 0.9970
Epoch 2: val_loss improved from 0.36421 to 0.17389, saving model to /home/emanoelcepr/Imagens/imagens_originais/Sentinel-2/treino_processadas/model70-30.h5
170/170 [==============================] - 158s 929ms/step - loss: 0.0995 - accuracy: 0.9970 - val_loss: 0.1739 - val_accuracy: 0.9615
Epoch 3/15
170/170 [==============================] - ETA: 0s - loss: 0.0715 - accuracy: 0.9974
Epoch 3: val_loss improved from 0.17389 to 0.09053, saving model to /home/emanoelcepr/Imagens/imagens_originais/Sentinel-2/treino_processadas/model70-30.h5
170/170 [==============================] - 164s 968ms/step - loss: 0.0715 - accuracy: 0.9974 - val_loss: 0.0905 - val_accuracy: 0.9889
Epoch 4/15
170/170 [==============================] - ETA: 0s - loss: 0.0541 - accuracy: 0.

Salva as métricas da rede já treinada

In [ ]:
# Salvar o objeto 'history' 
history_file = diretorio_imagens + "history.pkl"
with open(history_file, 'wb') as file:
    pickle.dump(history.history, file)


In [ ]:
print(history.history.keys())

Visualização dos Gráficos de Precisão e Perda

In [ ]:
def mostrar_graficos(history):
  fig = plt.gcf()
  fig.set_size_inches(16,4)
  plt.subplot(1,2,1)
  plt.plot(history.history['accuracy'], 'red', label='Accuracy treinamento')
  plt.plot(history.history['val_accuracy'], 'blue', label='Accuracy validação')
  plt.legend()
  plt.title('Accuracy')

  plt.subplot(1,2,2)
  plt.plot(history.history['loss'], 'red', label='Loss treinamento')
  plt.plot(history.history['val_loss'], 'blue', label='Loss validação')
  plt.legend()
  plt.title('Loss')
  
  plt.show()

mostrar_graficos(history)